In [1]:
%matplotlib inline
import geopandas as gpd
import numpy as np
import pandas as pd
import rioxarray
import xarray
from scipy.interpolate import interp2d
from tqdm import tqdm

# 1. Contaminantes: 


## Comunal

In [2]:
comunas = gpd.read_file("/Users/pipe/Documents/Spike/CR2/datos/mapas_censo/Comunas/comunas.shp")
comunas.columns = [m.lower() for m in comunas.columns]
comunas = comunas.to_crs({'init': 'epsg:4326'})
comunas['centroide_x'] = comunas.centroid.x
comunas['centroide_y'] = comunas.centroid.y
comunas.head(2)

,objectid,shape_leng,dis_elec,cir_sena,cod_comuna,codregion,st_area_sh,st_length_,region,comuna,provincia,geometry,centroide_x,centroide_y
0,48,170038.624165,16,8,6204,6,9.685774e+08,206184.271675,Región del Libertador Bernardo O'Higgins,Marchigüe,Cardenal Caro,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2...",-71.671741,-34.372586
1,29,125730.104795,15,8,6102,6,4.157446e+08,151911.576827,Región del Libertador Bernardo O'Higgins,Codegua,Cachapoal,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9...",-70.547220,-34.057048


In [3]:
path_base = '/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2'
emisiones = pd.read_csv(f'{path_base}/datos_streamlit/emisiones_a_nivel_comunal.csv',)

emisiones = emisiones.merge(comunas[['cod_comuna','comuna','provincia','centroide_x','centroide_y']],
                            on='comuna', how='left')
emisiones.head()

,región,comuna,CO,NH3,NO,NO2,NOX,O3,PM10,PM10ant,...,PM25bio,PMC,PMCPM10,SO2,SOA,pPPM,cod_comuna,provincia,centroide_x,centroide_y
0,0,Zona sin demarcar,0.066094,0.003609,0.000029,0.013203,0.013232,21.341276,2.570949,0.070516,...,3.843434,2.673085,0.408590,0.000199,3.050390e-07,0.000003,0,Zona sin demarcar,-73.256466,-49.486028
1,4,Andacollo,0.071681,0.025479,0.002716,0.146445,0.149161,23.543075,0.866612,0.705946,...,0.097107,0.439397,0.274630,0.679097,1.008164e-05,0.032183,4103,Elqui,-71.100631,-30.259459
2,4,Canela,0.094721,0.043335,0.001912,0.440166,0.442078,21.690690,1.186739,1.510238,...,0.235284,0.542326,0.214083,0.300273,1.666074e-05,0.869620,4202,Choapa,-71.394227,-31.401861
3,4,Combarbalá,0.072576,0.024044,0.001216,0.066119,0.067335,27.016204,0.831645,0.678423,...,0.098744,0.399148,0.268480,0.612323,4.771291e-06,0.013609,4302,Limarí,-70.965404,-31.146587
4,4,Coquimbo,0.091141,0.047380,0.056992,1.661522,1.718515,17.270764,2.061269,1.428171,...,0.700593,2.636373,0.395477,0.673434,3.183171e-05,0.721086,4102,Elqui,-71.358523,-30.228528


# 2. Socioeconomicos:
### Cargamos los datos socioeconomicos y los agregamos a los contaminantes 

In [69]:
# socioec = pd.read_csv('/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2/datos_streamlit/socioeconomicos_por_comuna.csv')
# columnas_que_queremos = socioec.columns[6:]
# socioec[columnas_que_queremos].head(2)

In [9]:
import pandas_gbq

socioec = pd.read_csv('/Users/pipe/Documents/Spike/CR2/datos/calidad_aire_2050_cr2/datos_streamlit/indicadores_CENSO17_por_comuna.csv')
socioec.columns = ['cod_comuna', 'personas','pueblo_indigena', 'inmigrantes']
consolidados = socioec.merge(comunas[['cod_comuna','comuna','provincia','region','centroide_x','centroide_y']], on='cod_comuna', how='left')
pandas_gbq.to_gbq(consolidados, 'CR2.habitantes_por_comuna', project_id='spike-sandbox')

# Guardamos los datos

In [71]:
#Consolidado comunal
consolidados.to_csv('/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2/datos_streamlit/emisiones_socioec_comunales.csv',index=False)

# 4. Centroide de cada región

In [72]:
lista_regiones = ['R01', 'R02', 'R03', 'R04', 'R05', 'R06',
               'R07', 'R08', 'R09', 'R10', 'R11', 'R12',
               'R13', 'R14', 'R15', 'R16',]
centroide_regiones = pd.DataFrame(columns = lista_regiones)
for region in lista_regiones:
    regiones = gpd.read_file("/Users/pipe/Documents/Spike/CR2/datos/mapas_censo/Regiones/shape_files/"+region+"/REGION_C17.shp")
    centroide_regiones[region] = [regiones.centroid.x[0], regiones.centroid.y[0]]

centroide_regiones = centroide_regiones.T
centroide_regiones.columns = ['centroide_x', 'centroide_y']
centroide_regiones.reset_index(inplace=True)
centroide_regiones['región'] = centroide_regiones['index'].apply(lambda x: int(x.replace('R0','').replace('R','')))


In [77]:
centroide_regiones.to_csv('/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2/datos_streamlit/centroide_regiones.csv',index=False)

In [73]:
#Consolidado regional
cols_emisiones = ['CO', 'NH3', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM10ant', 'PM10bio', 'PM25',
'PM25PM10', 'PM25ant', 'PM25bio', 'PMC','PMCPM10', 'SO2', 'SOA', 'pPPM',]
cols_demograficas = [ 'personas', 'pueblo_indigena', 'inmigrantes']
diccionario_agrupar = {m: 'sum' for m in cols_demograficas}
diccionario_agrupar.update({m: 'mean' for m in cols_emisiones})
consolidados_regional = consolidados.groupby(['región'])[cols_emisiones+cols_demograficas].agg(diccionario_agrupar)
consolidados_regional.reset_index(inplace=True)


In [74]:
consolidados_regional = consolidados_regional.merge(centroide_regiones[['centroide_x', 'centroide_y', 'región']],
                                         on='región', how='left' )

consolidados_regional = consolidados_regional.query('región>0')
consolidados_regional.reset_index().to_csv('/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2/datos_streamlit/emisiones_socioec_regionales.csv',index=False)

In [75]:
consolidados_regional.head()

,región,personas,pueblo_indigena,inmigrantes,CO,NH3,NO,NO2,NOX,O3,...,PM25PM10,PM25ant,PM25bio,PMC,PMCPM10,SO2,SOA,pPPM,centroide_x,centroide_y
1,4,757586.0,58194.0,10403.0,0.078107,0.028926,0.007178,0.336006,0.343184,24.650352,...,0.707648,0.815503,0.218268,1.243916,0.292352,0.539095,0.000012,0.256064,-70.860686,-30.619058
2,5,1807226.0,101647.0,28824.0,0.216279,0.095160,0.049404,5.949562,5.998966,17.533508,...,0.919837,6.865964,0.131654,0.263634,0.080163,0.047307,0.000041,6.541261,-71.387617,-32.731961
3,6,914555.0,50515.0,9613.0,0.224116,0.117723,0.084642,2.141902,2.226544,19.649391,...,0.955677,8.393967,0.094129,0.133963,0.044323,0.022780,0.000013,7.903341,-71.046549,-34.435383
4,7,1044950.0,39390.0,6714.0,0.215511,0.091944,0.023552,0.841823,0.865375,19.069366,...,0.926085,9.234161,0.115902,0.203629,0.073915,0.002963,0.000008,8.890575,-71.445584,-35.621334
5,8,1556805.0,157942.0,7347.0,0.175038,0.053106,0.059905,0.822983,0.882888,17.574946,...,0.919208,8.445629,0.358207,0.507270,0.080792,0.001269,0.000015,8.562118,-72.392403,-37.500578


In [28]:
### DEBUG

In [108]:
lista_variables_socio = [  'num_personas',  
  'is_34 porcentaje de viviendas con situación de allegamiento externo',
       'población total comunal luc_x', 'población usuaria de plazas',
       'bpu_28 porcentaje de población atendida por plazas públicas',
       'población total comunal luc_y', 'población usuaria de parques',
       'bpu_28 porcentaje de población atendida por parques públicas',
       'pob_urb_2017', 'pob_urb_afectada',
       'ea_48 porcentaje de población expuesta a inundación por tsunami',
       'superficie plazas (m2)', 'superficie de area verde (m2)',
       'poblacion cpv 2017',
       'bpu_29 superficie de area verde por habitantes (m2/hab)',
       'bpu_4 razón entre disponibilidad efectiva de matrículas y demanda potencial por educación',
       'bpu_3 distancia a e. basica (m)',
       'ea_22 consumo per capita residencial (kwh/persona)',
       'is_31 porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos',
       'is_40 porcentaje de manzanas con veredas con buena calidad de pavimento',
       'is_33 porcentaje de hacinamiento', 'población área de servicio_x',
       'superficie parques m²', 'bpu_23 superficie de parques por habitante',
       'bpu_20 distancia a plazas publicas (m)',
       'bpu_1 distancia a e. inicial (m)', 'población área de servicio_y',
       'superficie plazas m²', 'bpu_21 superficie de plazas por habitantes',
       'población 2017', 'lesionados 2018',
       'de_31 n° de lesionados en siniestros de tránsito por cada 100.000 habitantes',
       'poblacióm 2017', 'fallecidos 2018',
       'de_28 n° de víctimas mortales en siniestros de tránsito por cada 100.000 habitantes',
       'ig_90 porcentaje de participiación']

porcentajenan = {}
for var_socio in lista_variables_socio:
    porcentajenan[var_socio] = consolidados[var_socio].isna().sum()/consolidados.shape[0]

pd.DataFrame().from_dict(porcentajenan, orient='index')

,0
num_personas,0.003226
is_34 porcentaje de viviendas con situación de allegamiento externo,0.648387
población total comunal luc_x,0.648387
población usuaria de plazas,0.648387
bpu_28 porcentaje de población atendida por plazas públicas,0.648387
población total comunal luc_y,0.648387
población usuaria de parques,0.648387
bpu_28 porcentaje de población atendida por parques públicas,0.648387
pob_urb_2017,0.938710
pob_urb_afectada,0.938710


### Rescatamos las escalas